# AstraForge Toolkit - DeepAgent Ralph Mode (Notebook)

Run a Ralph-style autonomous loop using the DeepAgents Python API with the AstraForge sandbox backend.
Each iteration starts with fresh agent context, while `/workspace` acts as memory between runs.

Prereqs:
- Backend running at `http://localhost:8001` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install astraforge-toolkit`)
- A model provider (this example uses Ollama via `langchain-ollama`)


## Notebook walk-through

1. Install the toolkit, dotenv helpers, DeepAgents, and the Ollama runtime so the notebook can reach your local model provider.
2. Load `.env`, derive `BASE_URL`/`API_KEY`, and open a sandbox session that backs every iteration; the session ID is reprinted for continuity.
3. Create a `ChatOllama` LLM, wire it into `create_deep_agent`, and configure `SandboxBackend` plus the shell tool so DeepAgent can read/write `/workspace`.
4. `run_ralph_loop` uses fresh thread IDs while keeping `/workspace` as cross-iteration memory; it prints each iteration summary and returns the raw message history for inspection.
5. After iterating, the notebook flattens every message for richer inspection and shows how to stop the sandbox session when you finish.


### Visual flow overview

- Prepare the Ollama LLM, sandbox backend, and tools before kicking off Ralph iterations.
- Each loop prints an iteration header and captures the raw history so you can compare progress.
- The flattened messages cell renders an HTML view of every utterance for easier validation.
- Stop the sandbox session manually when you are done to keep `/workspace` tidy.


In [1]:
# Install dependencies (run from the examples/ folder)
%pip install astraforge-toolkit python-dotenv deepagents langchain-ollama --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

BASE_URL = os.getenv("ASTRA_FORGE_API_URL", "http://localhost:8001/api")
API_KEY = os.getenv("ASTRA_FORGE_API_KEY", "")  # for local setup go to http://localhost:5174/app/api-keys

if not API_KEY:
    raise RuntimeError("Set ASTRA_FORGE_API_KEY in your environment")

In [4]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
sandbox_session = client.create_sandbox_session()
SANDBOX_SESSION_ID = sandbox_session.session_id
print(f"Sandbox session: {SANDBOX_SESSION_ID} (workspace: {sandbox_session.workspace_path})")

Sandbox session: 846f8c66-1873-4c35-8933-83ff76e1828b (workspace: /workspace)


In [5]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gpt-oss:20b",
    validate_model_on_init=True,
    temperature=0,
    model_kwargs={"think": "high"},
)

In [6]:
from deepagents import create_deep_agent
from astraforge_toolkit import SandboxBackend, sandbox_shell

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        session_id=SANDBOX_SESSION_ID,
    )

tools = [sandbox_shell]

deep_agent = create_deep_agent(
    model=llm,
    backend=backend_factory,
    tools=tools,
)

In [7]:
import uuid

def _last_content(result):
    messages = result.get("messages", []) if isinstance(result, dict) else []
    if not messages:
        return result
    last = messages[-1]
    if hasattr(last, "content"):
        return last.content
    if isinstance(last, dict):
        return last.get("content", last)
    return last

def run_ralph_loop(task: str, max_iterations: int = 3):
    results = []
    iteration = 1
    while max_iterations == 0 or iteration <= max_iterations:
        thread_id = uuid.uuid4().hex  # fresh context each iteration
        run_config = {
            "thread_id": thread_id,
            "configurable": {"sandbox_session_id": SANDBOX_SESSION_ID},
        }
        iter_display = (
            f"{iteration}/{max_iterations}" if max_iterations > 0 else str(iteration)
        )
        prompt = f"""## Iteration {iter_display}

Your previous work is in the filesystem (/workspace/). Check what exists and keep building.

TASK:
{task}

Make progress. You'll be called again."""
        print(f"\n--- Ralph iteration {iter_display} ---")
        result = deep_agent.invoke(
            {"messages": [{"role": "user", "content": prompt}]},
            config=run_config,
        )
        results.append(result)
        print(_last_content(result))
        iteration += 1
    return results

In [8]:
TASK = "Build a tiny checklist app in /workspace and keep iterating on it."
MAX_ITERATIONS = 2

results = run_ralph_loop(TASK, MAX_ITERATIONS)


--- Ralph iteration 1/2 ---


backend response to llm: Error: String not found in file: 'body {
  font-family: Arial, sans-serif;
  margin: 20px;
}

h1 {
  text-align: center;
}

#addForm {
  display: flex;
  justify-content: center;
  margin-bottom: 20px;
}

#newItem {
  width: 60%;
  padding: 8px;
  font-size: 1rem;
}

button {
  padding: 8px 12px;
  margin-left: 8px;
  font-size: 1rem;
}
ul {
  list-style: none;
  padding: 0;
}
li {
  padding: 8px;
  border-bottom: 1px solid #ddd;
}
'
backend response to llm: Error: String not found in file: 'document.addEventListener('DOMContentLoaded', () => {
  const form = document.getElementById('addForm');
  const list = document.getElementById('list');
  const input = document.getElementById('newItem');

  form.addEventListener('submit', (e) => {
    e.preventDefault();
    const text = input.value.trim();
    if (!text) return;
    addItem(text);
    input.value = '';
  });

  function addItem(text) {
    const li = document.createElement('li');
    li.textContent = text

All requested changes have been applied. The script now supports checkboxes, delete buttons, and completed styling, and the CSS has been updated accordingly.

--- Ralph iteration 2/2 ---
**Checklist app updated**

- Added persistence: items are loaded from `localStorage` on page load.
- `addItem` now accepts a `completed` flag and updates the UI accordingly.
- All changes (add, delete, toggle) trigger `saveItems()` to keep `localStorage` in sync.
- The app now remembers your checklist across page reloads.

You can open `/workspace/checklist/index.html` in a browser to see the working app. Let me know what you'd like to add next!


In [15]:
flat_messages = [msg for item in results for msg in item['messages']]

for m in flat_messages:
    print(m.pretty_repr(html=True))

================================ Human Message =================================

## Iteration 1/2

Your previous work is in the filesystem (/workspace/). Check what exists and keep building.

TASK:
Build a tiny checklist app in /workspace and keep iterating on it.

Make progress. You'll be called again.
================================== Ai Message ==================================
Tool Calls:
  ls (61d89640-feb8-490f-9e11-18783ca4ad2f)
 Call ID: 61d89640-feb8-490f-9e11-18783ca4ad2f
  Args:
    path: /workspace
================================= Tool Message =================================
Name: ls

['/workspace/.', '/workspace/..']
================================== Ai Message ==================================
Tool Calls:
  write_file (ce0258b1-7bb0-43c9-9704-72bc690c39ab)
 Call ID: ce0258b1-7bb0-43c9-9704-72bc690c39ab
  Args:
    content: <!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <ti

In [16]:
# Optional: stop the sandbox session when you are done
client.stop_sandbox_session(SANDBOX_SESSION_ID)